# How to do a Monte-Carlo simulation of failure times with lsdfailtools 

This notebook shows how to set up the range of parameters and run a given number of simulation on parallel cores

## First let's import the model and the rainfall data

In [1]:
# Importing the model
import lsdfailtools.iverson2000 as iverson
# I'll need that to process the outputs
import pandas as pd
import numpy as np

# rainfall data
df = pd.read_csv("preprocessed_data.csv")

# Setting the depth resolution vector
depths = np.arange(0.2,3.1,0.1)


## Now let's set up the parameter ranges

The function call is a bit chunky but simply add the different minimum and maximum values for eahc parameter and the code will randomly sample values within it.

Note that `D_0`and `K_sat_min` are sampled in log space and the other are uniform. The depth resolution needs to be the same for each siumulation.

In [2]:
MCrun = iverson.MonteCarlo_Iverson( alpha_min = 0.1, D_0_min = 1e-6,K_sat_min = 1e-8, d_min = 0.5,Iz_over_K_steady_min = 0.1,
      friction_angle_min = 0.2, cohesion_min = 5000, weight_of_water_min = 9800, 
      weight_of_soil_min = 15000, alpha_max = 0.8, D_0_max = 1e-4,K_sat_max = 1e-6, d_max = 3,Iz_over_K_steady_max = 0.8,
      friction_angle_max = 0.5, cohesion_max = 20000, weight_of_water_max = 9801, 
      weight_of_soil_max = 25000, depths = depths)

## Now let's run the simulations

The simulation are run for a given set of precipitation record. The simulation can take a lot of time, it saves the output into a csv every 20 simulations if you need to look at intermediate resulsts.

In [3]:
MCrun.run_MC_failure_test(df["duration_s"].values, df["intensity_mm_sec"].values,
                          n_process = 2, output_name = "test_MC.csv", n_iterations = 100, replace = True)


## Now Checking the output

Let's load the csv filecontaining the result. The column "time_of_failure" contains the ... time of failure. -9999 Means no failure and 0. means failure from scratch.

In [5]:
res = pd.read_csv("test_MC.csv")
res["time_of_failure"]

0      9942329.0
1        -9999.0
2     13722434.0
3        -9999.0
4        -9999.0
         ...    
95           0.0
96       -9999.0
97       -9999.0
98     9719127.0
99       -9999.0
Name: time_of_failure, Length: 100, dtype: float64